# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.10.57 openai==1.59.8 llama-index-finetuning==0.1.12 llama-index-embeddings-huggingface==0.2.3 llama-index-embeddings-cohere==0.1.9 llama-index-readers-web==0.1.23 cohere==5.6.2 tiktoken==0.8.0 chromadb==0.5.5 html2text==2024.2.26 sentence-transformers==2.7.0 pydantic==2.10.5 llama-index-vector-stores-chroma==0.1.10 kaleido==0.2.1 llama-index-llms-gemini==0.1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 30.2

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR-GOOGLE-API-KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

In [3]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.
import nest_asyncio

nest_asyncio.apply()

# Load a Model


In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


In [5]:
from huggingface_hub import hf_hub_download

vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip", repo_type="dataset", local_dir=".")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [6]:
!unzip -o vectorstore.zip

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [7]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

# Create the vector index
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
vector_index = VectorStoreIndex.from_vector_store(vector_store)

# Multi-Step Query Engine


## GPT-4o-mini


In [12]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

step_decompose_transform_gpt4o = StepDecomposeQueryTransform(verbose=True, llm=Settings.llm)

In [57]:
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine

#Default query engine
query_engine_gpt4o_mini = vector_index.as_query_engine()

# Multi Step Query Engine
multi_step_query_engine = MultiStepQueryEngine(
    query_engine = query_engine_gpt4o_mini,
    query_transform = step_decompose_transform_gpt4o,
    index_summary = "Used to answer the Questions about RAG, Machine Learning, Deep Learning, and Generative AI, Note: Don't repeat the Same quesion",
)

# Query Dataset

## Default

In [58]:
# Default query engine
query_engine = vector_index.as_query_engine()
res = query_engine.query("Write about Llama 3.1 Model, BERT and PEFT methods")
print(res.response)

The Llama model, particularly in its adaptations like Llama-Adapter, focuses on efficient fine-tuning techniques to enhance its capabilities, such as transforming it into an instruction-following model. This is achieved through the use of adaptation prompts that are integrated into the model while keeping its pre-trained knowledge intact. The Llama-Adapter method introduces a minimal number of learnable parameters, making it a lightweight solution for fine-tuning.

BERT, another foundational model in natural language processing, employs a different architecture and training approach, primarily focusing on masked language modeling and next sentence prediction. While BERT is designed for a variety of NLP tasks, it typically requires more extensive fine-tuning compared to methods like Llama-Adapter, which emphasizes parameter efficiency.

The PEFT (Parameter-Efficient Fine-Tuning) methods, including LoRA and Llama-Adapter, are designed to adapt large models like Llama and BERT to specific

In [59]:
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 781b7b12-eca2-47c0-a66e-9d6be670e951
Title	 LLaMA
Text	 on how to fine-tune LLaMA model using LoRA method via the 🤗 PEFT library with intuitive UI. 🌎 - A [notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/text-generation-open-llama.ipynb) on how to deploy Open-LLaMA model for text generation on Amazon SageMaker. 🌎 ## LlamaConfig[[autodoc]] LlamaConfig## LlamaTokenizer[[autodoc]] LlamaTokenizer    - build_inputs_with_special_tokens    - get_special_tokens_mask    - create_token_type_ids_from_sequences    - save_vocabulary## LlamaTokenizerFast[[autodoc]] LlamaTokenizerFast    - build_inputs_with_special_tokens    - get_special_tokens_mask    - create_token_type_ids_from_sequences    - update_post_processor    - save_vocabulary## LlamaModel[[autodoc]] LlamaModel    - forward## LlamaForCausalLM[[autodoc]] LlamaForCausalLM    - forward## LlamaForSequenceClassification[[autodoc]] LlamaForSequenceClassif

## GPT-4o-mini Multi-Step


In [60]:
response = multi_step_query_engine.query("Write about Llama 3.1 Model, BERT and PEFT methods")
print(response.response)

> Current query: Write about Llama 3.1 Model, BERT and PEFT methods
> New query: What are the key features of the Llama 3.1 Model?
> Current query: Write about Llama 3.1 Model, BERT and PEFT methods
> New query: What are the key features of the Llama 3.1 Model?
> Current query: Write about Llama 3.1 Model, BERT and PEFT methods
> New query: What are the key features of BERT?
The Llama 3.1 model is a state-of-the-art language model developed by Meta, featuring significant advancements in scale and capabilities. It is the largest model from Meta, trained on over 15 trillion tokens using more than 16,000 H100 GPUs. One of its standout features is the extended context length of 128K, which allows it to process and understand longer texts effectively. The model demonstrates enhanced reasoning and coding abilities, excels in generating high-quality code, and showcases strong logical reasoning and problem-solving skills. Additionally, it supports zero-shot tool use and has a multilingual proc

In [61]:
for query, response in response.metadata['sub_qa']:
    print(f"**{query}**\n{response}\n")

**What are the key features of the Llama 3.1 Model?**
The Llama 3.1 model boasts several key features, including:

1. **Model Scale and Training**: It is the largest model from Meta, trained on over 15 trillion tokens using more than 16,000 H100 GPUs.

2. **Extended Context Length**: The model supports a context length of 128K, enhancing its ability to handle longer inputs.

3. **Enhanced Capabilities**: It demonstrates improved reasoning and coding abilities compared to its predecessors.

4. **Tool Use and Agentic Behaviors**: The model supports zero-shot tool use, allowing it to perform tasks without prior training on specific tools.

5. **Multilingual Processing**: Approximately 50% of its training data consists of multilingual tokens, enabling effective understanding and processing of multiple languages.

6. **Programming and Reasoning Skills**: Llama 3.1 excels in generating high-quality code and exhibits strong logical reasoning, problem-solving, and analytical skills.

7. **Benc

In [62]:
for src in response.source_nodes:
    print("Node ID\t", src.node_id)
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 a6ebe22b-5529-4e78-93a6-e7336d031600
Text	 GPT-2  GPT-3  and GPT-4  which were decoder-only architectures. Another well-known example is BERT (Bidirectional Encoder Representations from Transformers)  an encoder-only transformer mode used as a component in sentence embedding models.   Lets talk about BERT!BERT stands for Bidirectional Encoder Representations from Transformers. It is a language model by Google that uses a transformer architecture to understand and generate human-like language. BERT is designed to simultaneously process text in both directions  allowing it to capture context more effectively than traditional unidirectional models  which read text sequentially from left to right or right to left.   Example of Bidirectional CapabilityConsider the sentence:    The bank is situated on the _______ of the river.   In a unidirectional model  understanding the blank would primarily rely on the words before it  potentially leading to ambiguity about whether bank refers t

# Test gemini-1.5-flash Multi-Step


In [43]:
from llama_index.core import ServiceContext
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine

from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-1.5-flash")

service_context_gemini = ServiceContext.from_defaults(llm=llm)

step_decompose_transform = StepDecomposeQueryTransform(llm=llm, verbose=True)

query_engine_gemini = vector_index.as_query_engine(
    service_context=service_context_gemini
)
query_engine_gemini = MultiStepQueryEngine(
    query_engine=query_engine_gemini,
    query_transform=step_decompose_transform,
    index_summary="Used to answer the Questions about RAG, Machine Learning, Deep Learning, and Generative AI. Note: Don't repeat the Same question",
)

<ipython-input-43-46371d5420d8>:11: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gemini = ServiceContext.from_defaults(llm=llm)


In [44]:
response_gemini = query_engine_gemini.query("Write about Llama 3.1 Model, BERT and PEFT")

> Current query: Write about Llama 3.1 Model, BERT and PEFT
> New query: None



In [45]:
response_gemini.response

'Empty Response'

## Test Retriever on Multistep


In [46]:
# import llama_index
# from llama_index.core.indices.query.schema import QueryBundle

# t = QueryBundle("How Retrieval Augmented Generation (RAG) work?")
# query_engine_gemini.retrieve(t)

## Subquestion Query Engine

In [47]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = vector_index.as_query_engine()

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="LlamaIndex",
            description="Used to answer the Questions about RAG, Machine Learning, Deep Learning, and Generative AI. Note: Don't repeat the Same question",
        ),
    ),
]

sub_question_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

response = sub_question_engine.query("Write about Llama 3.1 Model, BERT and PEFT")


Generated 5 sub questions.
[LlamaIndex] Q: What are the key features and improvements of the Llama 3.1 model compared to its predecessors?
[LlamaIndex] Q: How does the BERT model work and what are its main applications in natural language processing?
[LlamaIndex] Q: What is PEFT (Parameter-Efficient Fine-Tuning) and how does it enhance the performance of models like BERT and Llama 3.1?
[LlamaIndex] Q: What are the differences in architecture between Llama 3.1 and BERT?
[LlamaIndex] Q: In what scenarios is PEFT particularly beneficial for fine-tuning language models?
[LlamaIndex] A: The provided information does not detail the architectural differences between Llama 3.1 and BERT. It primarily focuses on the specifications, performance, and advantages of Llama 3.1, particularly its hardware requirements, benchmarking results, and open-source benefits. For a comprehensive comparison of architectures, additional resources would be needed.
[LlamaIndex] A: PEFT is particularly beneficial for

In [48]:
response.response

'Llama 3.1 is a state-of-the-art language model developed by Meta, notable for being the largest model in its series, trained on over 15 trillion tokens with the help of more than 16,000 H100 GPUs. This extensive training enhances its capabilities significantly. Key features of Llama 3.1 include support for a 128K context length, improved reasoning and coding abilities, multilingual processing, and the capacity for zero-shot tool use and agentic behaviors. It also demonstrates superior performance in benchmark tests, particularly in mathematical reasoning and long text processing.\n\nBERT, on the other hand, utilizes a transformer architecture that processes text bidirectionally, allowing it to capture context more effectively than traditional models. It is pre-trained on tasks such as Masked Language Modeling and Next Sentence Prediction, which enable it to understand relationships between words and sentences. BERT is widely applied in natural language processing tasks, including sent

# HyDE Transform


In [49]:
query_engine = vector_index.as_query_engine()

In [50]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine.transform_query_engine import TransformQueryEngine

hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [51]:
response = hyde_query_engine.query("Write about Llama 3.1 Model, BERT and PEFT")

In [52]:
response.response

'Llama 3.1 405B is a significant advancement in the field of AI, developed by Meta. It stands out as the largest open-source model to date, trained on over 15 trillion tokens using more than 16,000 H100 GPUs. This extensive training has enabled it to achieve a remarkable 128K context length, which enhances its ability to handle complex reasoning and long document summarization. The model has been designed to outperform proprietary models like GPT-4o and Claude 3.5 Sonnet in various benchmark tests, particularly in areas such as general knowledge, steerability, mathematical reasoning, and multilingual processing.\n\nIn terms of features, Llama 3.1 includes enhanced reasoning and coding capabilities, allowing it to generate high-quality code and perform well in programming tasks. It also supports multilingual processing, with about 50% of its training data consisting of multilingual tokens, enabling effective understanding and processing of multiple languages.\n\nBERT, or Bidirectional E

In [53]:
for src in response.source_nodes:
    print("Node ID\t", src.node_id)
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 5624cdc8-2997-4e4d-82d1-c7383d389215
Text	 3.1 405B is Metas largest model  trained with over 15 trillion tokens. For this  Meta optimized the entire training stack and trained it on more than 16 000 H100 GPUs  making it the first Llama model trained at this scale.   According to Meta  this version of the original model (Llama 1 and Llama 2) has 128K context length  improved reasoning and coding capabilities. Meta has also upgraded both multilingual 8B and 70B models.   Key Features of Llama 3.1 40 5B:Llama 3.1 comes with a host of features and capabilities that appeal to The users  such as:   RAG & tool use  Meta states that you can use Llama system components to extend the model using zero-shot tool use and build agentic behaviors with RAG.   Multi-lingual  Llama 3 naturally supports multilingual processing. The pre-training data includes about 50% multilingual tokens and can process and understand multiple languages.   Programming and Reasoning  Llama 3 has powerful program

In [54]:
query_bundle = hyde("Write about Llama 3.1 Model, BERT and PEFT")

In [55]:
hyde_doc = query_bundle.embedding_strs[0]

In [56]:
hyde_doc

"The Llama 3.1 model, developed by Meta, represents a significant advancement in the field of natural language processing (NLP). It builds upon the foundation laid by its predecessors, Llama 1 and Llama 2, by incorporating more extensive training data and improved architectural designs. Llama 3.1 is designed to enhance performance in various NLP tasks, such as text generation, summarization, and question-answering, while also being more efficient in terms of computational resources. This model is particularly notable for its ability to generate coherent and contextually relevant text, making it a valuable tool for developers and researchers in the AI community.\n\nIn contrast, BERT (Bidirectional Encoder Representations from Transformers), introduced by Google in 2018, revolutionized the way models understand language by employing a bidirectional approach to context. Unlike traditional models that read text sequentially, BERT processes words in relation to all the other words in a sent